# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [10]:
# Write your code below.

%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [11]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [12]:
import os
from glob import glob

price_data = glob(os.path.join(os.getenv('PRICE_DATA'), "*/*/*.parquet"), recursive = True) # glob adds all files into a list
price_data

['../../05_src/data/prices\\ACIU\\ACIU_2016\\part.0.parquet',
 '../../05_src/data/prices\\ACIU\\ACIU_2016\\part.1.parquet',
 '../../05_src/data/prices\\ACIU\\ACIU_2017\\part.0.parquet',
 '../../05_src/data/prices\\ACIU\\ACIU_2017\\part.1.parquet',
 '../../05_src/data/prices\\ACIU\\ACIU_2018\\part.0.parquet',
 '../../05_src/data/prices\\ACIU\\ACIU_2018\\part.1.parquet',
 '../../05_src/data/prices\\ACIU\\ACIU_2019\\part.0.parquet',
 '../../05_src/data/prices\\ACIU\\ACIU_2019\\part.1.parquet',
 '../../05_src/data/prices\\ACIU\\ACIU_2020\\part.0.parquet',
 '../../05_src/data/prices\\ACIU\\ACIU_2020\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet',
 '../../05_src/data/

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
import dask.dataframe as dd

df = dd.read_parquet(price_data[0:10]) # computer memory issues, only loading a sliver of the data so that I can get it to run on my end!

In [21]:
# Create lags columns

df['Close_lag'] = df.groupby(['ticker'])['Close'].shift(1)
df['Adj_Close_lag'] = df.groupby(['ticker'])['Adj Close'].shift(1)

C:\Users\Andrea\AppData\Local\Temp\ipykernel_2164\2936401372.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df['Close_lag'] = df.groupby(['ticker'])['Close'].shift(1)
C:\Users\Andrea\AppData\Local\Temp\ipykernel_2164\2936401372.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df['Adj_Close_lag'] = df.groupby(['ticker'])['Adj Close'].shift(1)


In [22]:
# Create Returns column: Close / Close_lag - 1

df['returns'] = df['Close']/df['Close_lag'] - 1
df

,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag,Adj_Close_lag,returns
npartitions=10,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,object,object,int32,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [23]:
# Create High-Low Range column: High - Low

df['hi_lo_range'] = df['High']-df['Low']
df

,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag,Adj_Close_lag,returns,hi_lo_range
npartitions=10,,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,object,object,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [24]:
# Assign dataframe to dd_feat

dd_feat = df
dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag,Adj_Close_lag,returns,hi_lo_range
npartitions=10,,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,object,object,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [25]:
# Convert Dask dataframe to Pandas dataframe

pd_df = dd_feat.compute()

In [27]:
# Perform Rolling Average calculation

pd_df.groupby('ticker').apply(
    lambda x: x.assign(Moving_Avg = x['returns'].rolling(10).mean())
)

C:\Users\Andrea\AppData\Local\Temp\ipykernel_2164\53279683.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pd_df.groupby('ticker').apply(


Date  Open  High    Low  Close  Adj Close  Volume    source  \
ticker                                                                         
ACIU   823 2020-01-02  8.32  8.77  8.100   8.66       8.66  279000  ACIU.csv   
       825 2020-01-06  8.54  9.00  8.450   8.99       8.99  161500  ACIU.csv   
       830 2020-01-13  9.27  9.52  9.032   9.40       9.40  129900  ACIU.csv   
       840 2020-01-28  8.68  8.85  8.590   8.80       8.80   52200  ACIU.csv   
       853 2020-02-14  8.47  8.64  8.385   8.52       8.52  148500  ACIU.csv   
...               ...   ...   ...    ...    ...        ...     ...       ...   
       653 2019-05-01  4.77  4.92  4.680   4.80       4.80  108500  ACIU.csv   
       667 2019-05-21  4.84  5.05  4.840   5.03       5.03  114700  ACIU.csv   
       682 2019-06-12  5.26  5.98  5.260   5.84       5.84  503400  ACIU.csv   
       687 2019-06-19  5.77  5.79  5.670   5.79       5.79   46400  ACIU.csv   
       694 2019-06-28  5.42  5.56  5.370   5.55       5.55   51400  ACIU.csv   

           ticker  Year  Close_lag  Adj_Close_lag   returns  hi_lo_range  \
ticker                                                                     
ACIU   823   ACIU  2020       8.52           8.52  0.016432     0.670000   
       825   ACIU  2020       8.68           8.68  0.035714     0.550000   
       830   ACIU  2020       9.30           9.30  0.010753     0.488001   
       840   ACIU  2020       8.71           8.71  0.010333     0.260000   
       853   ACIU  2020       8.45           8.45  0.008284     0.255000   
...           ...   ...        ...            ...       ...          ...   
       653   ACIU  2019       4.76           4.76  0.008403     0.240000   
       667   ACIU  2019       4.89           4.89  0.028630     0.210000   
       682   ACIU  2019       5.25           5.25  0.112381     0.720000   
       687   ACIU  2019       5.74           5.74  0.008711     0.120000   
       694   ACIU  2019       5.44           5.44  0.020221     0.190000   

            Moving_Avg  
ticker                  
ACIU   823         NaN  
       825         NaN  
       830         NaN  
       840         NaN  
       853         NaN  
...                ...  
       653   -0.003597  
       667   -0.000622  
       682    0.015060  
       687    0.019143  
       694    0.015368  

[886 rows x 15 columns]

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Answer: We do not need to convert it to pandas to run the calculation. Dask provides us a more efficient way to run the calculations.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.